In [1]:
pip install transformers accelerate peft datasets bitsandbytes

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.3 MB/s eta 0:00:00


In [2]:
!pip install rouge-score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=58aff16b79df00806edf3cf26a9000b0a2bd656ec278f2c348e651e4ac2c7a28
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [3]:
import pandas as pd
from transformers import (
    MarianMTModel, MarianTokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments
)
from peft import LoraConfig, get_peft_model, PeftModel
from datasets import Dataset
import nltk
from rouge_score import rouge_scorer
import numpy as np

# BLEU Score Calculation Function
def CalculateAvgBLEUScore(Transcribed_Sentences, Validated_Sentences, tokenizer):
    BLEU_Scores = []
    for i in range(len(Transcribed_Sentences)):
        Valid_Sentence = tokenizer.tokenize(Validated_Sentences[i])
        Transcribed_Sentence = tokenizer.tokenize(Transcribed_Sentences[i])
        BLEU_Scores.append(nltk.translate.bleu_score.sentence_bleu([Valid_Sentence], Transcribed_Sentence))
    return np.mean(BLEU_Scores)


In [4]:
# ROUGE Score Calculation Function
def CalculateAvgROUGEScore(Transcribed_Sentences, Validated_Sentences):
    rs = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    ROUGE1_Precision_Scores = []
    ROUGE1_Recall_Scores = []
    ROUGE1_F1_Scores = []
    ROUGE2_Precision_Scores = []
    ROUGE2_Recall_Scores = []
    ROUGE2_F1_Scores = []
    ROUGEL_Precision_Scores = []
    ROUGEL_Recall_Scores = []
    ROUGEL_F1_Scores = []
    for i in range(len(Transcribed_Sentences)):
        rouge_score = rs.score(Validated_Sentences[i], Transcribed_Sentences[i])
        ROUGE1_Precision_Scores.append(rouge_score['rouge1'][0])
        ROUGE1_Recall_Scores.append(rouge_score['rouge1'][1])
        ROUGE1_F1_Scores.append(rouge_score['rouge1'][2])
        ROUGE2_Precision_Scores.append(rouge_score['rouge2'][0])
        ROUGE2_Recall_Scores.append(rouge_score['rouge2'][1])
        ROUGE2_F1_Scores.append(rouge_score['rouge2'][2])
        ROUGEL_Precision_Scores.append(rouge_score['rougeL'][0])
        ROUGEL_Recall_Scores.append(rouge_score['rougeL'][1])
        ROUGEL_F1_Scores.append(rouge_score['rougeL'][2])
    return {
        "rouge1": [np.mean(ROUGE1_Precision_Scores), np.mean(ROUGE1_Recall_Scores), np.mean(ROUGE1_F1_Scores)],
        "rouge2": [np.mean(ROUGE2_Precision_Scores), np.mean(ROUGE2_Recall_Scores), np.mean(ROUGE2_F1_Scores)],
        "rougeL": [np.mean(ROUGEL_Precision_Scores), np.mean(ROUGEL_Recall_Scores), np.mean(ROUGEL_F1_Scores)]
    }

In [5]:
# Dataset preparation
df = pd.read_csv("hf://datasets/salehalmansour/english-to-arabic-translate/en_ar_final.tsv", sep="\t")
df = df[['en', 'ar']]
df = df.head(50000)
dataset = Dataset.from_pandas(df)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [6]:
# Load model and tokenizer
model_name = "Helsinki-NLP/opus-mt-en-ar"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

lora_config = LoraConfig(
    r=2,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none"
)

model = get_peft_model(model, lora_config)

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/801k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/917k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.12M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [7]:
# Tokenize the dataset
def tokenize_function(examples):
    inputs = tokenizer(examples['en'], truncation=True, padding="max_length", max_length=128)
    targets = tokenizer(examples['ar'], truncation=True, padding="max_length", max_length=128)
    inputs['labels'] = targets['input_ids']
    return inputs

tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [18]:
# Training configuration
training_args = Seq2SeqTrainingArguments(
    output_dir="./lora-opus-mt-en-ar",
    evaluation_strategy="steps",
    eval_steps=500,
    learning_rate=3e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    save_total_limit=1,
    save_steps=1000,
    num_train_epochs=1,
    predict_with_generate=True,
    logging_dir="./logs",
    logging_steps=100,
    report_to="none",
    fp16=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,

)

# Train the model
trainer.train()

# Save the model and tokenizer
model.save_pretrained("./lora-opus-mt-en-ar")
tokenizer.save_pretrained("./lora-opus-mt-en-ar")

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Step,Training Loss,Validation Loss
500,0.175600,0.154431
1000,0.164000,0.132012
1500,0.142800,0.116407


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[62801]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[62801]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strate

('./lora-opus-mt-en-ar/tokenizer_config.json',
 './lora-opus-mt-en-ar/special_tokens_map.json',
 './lora-opus-mt-en-ar/vocab.json',
 './lora-opus-mt-en-ar/source.spm',
 './lora-opus-mt-en-ar/target.spm',
 './lora-opus-mt-en-ar/added_tokens.json')

In [19]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv('/content/Translated_data.csv')

# Print the DataFrame to verify
print(df.head())

                                             English  \
0   She hides her grief and joins in the homecomi...   
1   Rokshik stayed at the palace of his personal ...   
2                      She had an uneventful career.   
3                   Let's eat the chocolate tonight.   
4                      Newport was a county borough.   

                                              Arabic  
0    إنها تخفي حزنها وتنضم إلى احتفالات العودة للوطن  
1  ومكث روكشيك في قصر صديقه الشخصي، الدكتاتور فير...  
2                  كان لديها حياة وظيفية غير مستقرة.  
3                        دعنا نأكل الشوكولاتة الليلة  
4                             (نيوبورت) كانت مقاطعة.  


In [20]:
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer

# Load the CSV file with English sentences
df = pd.read_csv('/content/Translated_data.csv')

# If the CSV contains both English and Arabic, you can access them directly
# english_sentences = df['English'].tolist()

# Otherwise, extract English sentences
english_sentences = df['English'].tolist()

# Load pre-trained MarianMT model and tokenizer for English-Arabic translation
model_name = 'Helsinki-NLP/opus-mt-en-ar'  # Example model for English-Arabic translation
model = MarianMTModel.from_pretrained(model_name)
tokenizer = MarianTokenizer.from_pretrained(model_name)

# Translate English sentences to Arabic
arabic_sentences = english_sentences.copy()
index = 0
for sentence in english_sentences:
    print(index)
    input_ids = tokenizer(sentence, return_tensors="pt").input_ids
    output_ids = model.generate(input_ids)
    translated_sentence = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    arabic_sentences[index] = translated_sentence
    index += 1

# Saving the translated data to a new CSV file
df_translated = pd.DataFrame({'English': english_sentences, 'Arabic': arabic_sentences})

# Save the new CSV file with the translations
df_translated.to_csv('/content/Translated_data_with_translations.csv', index=False)

# Load the newly saved CSV file to verify
df_loaded = pd.read_csv('/content/Translated_data_with_translations.csv')

# Verify by printing some rows
print(df_loaded.head())


/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [21]:
# BLEU & ROUGE Scoring
validated_arabic_sentences = df['Arabic'].tolist()  # The validated Arabic translations from the dataset
transcribed_arabic_sentences = arabic_sentences  # The machine-generated Arabic translations

# Calculate BLEU Score
bleu_score = CalculateAvgBLEUScore(transcribed_arabic_sentences, validated_arabic_sentences, tokenizer)
print(f"Average BLEU Score: {bleu_score}")

# Calculate ROUGE Score
rouge_scores = CalculateAvgROUGEScore(transcribed_arabic_sentences, validated_arabic_sentences)
print("               Precision           | Recall             | F1")
print(f"Rouge_1 Score: {rouge_scores['rouge1']}")
print(f"Rouge_2 Score: {rouge_scores['rouge2']}")
print(f"Rouge_L Score: {rouge_scores['rougeL']}")

Average BLEU Score: 1.0
               Precision           | Recall             | F1
Rouge_1 Score: [0.03680981595092025, 0.03680981595092025, 0.03680981595092025]
Rouge_2 Score: [0.018404907975460124, 0.018404907975460124, 0.018404907975460124]
Rouge_L Score: [0.03680981595092025, 0.03680981595092025, 0.03680981595092025]
